# DI challenge q1

Overall view

1. Matrix to represent status of the towers
2. At each step, methods to search available moves
3. Iteration to perform moves
4. Methods to determine conversion
5. Output

6'. time count


In [45]:
import numpy as np
import os
import random

In [387]:
class TowerMatrix:
    """Matrix to represent status of the tower"""
    def __init__(self,**kwargs):
        self.M, self.N = 3,3
        #matrix(M,N), M number of stacks, N number of disks
        if 'M' in kwargs:
            self.M = kwargs['M']
        if 'N' in kwargs:
            self.N = kwargs['N']
        self.status = np.zeros((self.M,self.N))
        self.status[0] = np.arange(1,self.N+1)
        self.denominator =  (1+self.N)*self.N/2.0
        self.PossibleMove = []
        #print(self.status[0])
        #print(self.status)
    def reset(self):
        """reset matrix to initial status"""
        self.status = np.zeros((self.M,self.N))
        self.status[0] = np.arange(1,self.N+1)       
    def move(self):
        """general method for move
            output self.status after move"""
        #generate available move list
        self.SearchMove()
        #randomly chooseone
        #move_pair = random.SystemRandom().choice(self.PossibleMove)
        move_pair = self.PossibleMove[np.random.choice(len(self.PossibleMove))]
        self.move_by_pair(move_pair)
    
    def move_by_pair(self, move_pair):
        self.status[move_pair[0]], self.status[move_pair[1]] = self.status[move_pair[1]], self.status[move_pair[0]]
        
    def SearchMove(self):
        self.PossibleMove = []
        """method to search movesm, return a """
        #current_disk,current_index = self.TopDisk(self.status[0])
        next_disk,next_index = self.TopDisk(self.status[0])
        #disk_to_move = self.Compare(current_disk,current_index,next_disk,next_index)
        #if disk_to_move == 1:
        #    self.PossibleMove.append([(0,current_index),(1,next_index-1)])
        #if disk_to_move == 2:
        #    self.PossibleMove.append([(0,current_index-1),(1,next_index)])
        for i in range(self.M-1):
            current_disk,current_index = next_disk,next_index 
            next_disk,next_index = self.TopDisk(self.status[i+1])
            disk_to_move = self.Compare(current_disk,next_disk)
            #print i
            if disk_to_move == 1:
                self.PossibleMove.append([(i,current_index),(i+1,next_index-1)])
            if disk_to_move == 2:
                self.PossibleMove.append([(i,current_index-1),(i+1,next_index)])
                
    def TopDisk(self, stack):
        """method to generate top disk and its index for one stack, return as diskmass,index; if no disk return 0, self.N"""
        disk_mass = 0
        index = self.N
        for i in range(self.N):
            if stack[i] != 0:
                index = i
                disk_mass = stack[i]
                break
        return disk_mass,index
    
    def Compare(self,disk_mass1,disk_mass2):
        """Method to compare two stack top, return '0' for no swapping, '1' for 1 to 2, '2' for 2 to 1"""
        if disk_mass1 + disk_mass2 == 0:
            return 0
        else:
            if disk_mass1*disk_mass2 == 0:
                if disk_mass1 == 0:
                    return 2
                if disk_mass2 == 0:
                    return 1
            else:
                if disk_mass1 > disk_mass2:
                    return 2
                else:
                    return 1
                
    def COM(self):
        """Calculate COM"""
        numerator = 0.0
        for i in range(self.M):
            numerator += (i+1.0) * np.sum(self.status[i])
        return  numerator/self.denominator
    
    def set_status(self,status):
        self.status = status


In [388]:
#New thoughts, calculate probability 
M=3
N=3
Initial = TowerMatrix(M=M,N=N)
T = 16

Stat_list_next = [Initial.status]

Prob_list_next = [1.0]

temp_tower = TowerMatrix(M=M,N=N)
next_tower = TowerMatrix(M=M,N=N)
for i in range(T):
    Stat_list = Stat_list_next
    Prob_list = Prob_list_next
    Stat_list_next = []
    Prob_list_next = []
    #loop over status
    print ("T=",i)
    print (len(Stat_list),len(Prob_list))
    for state, prob in zip(Stat_list,Prob_list):
        temp_tower.reset()
        temp_tower.set_status(state)
        print("next_tower",state)
        #print("COM",temp_tower.COM())
        temp_tower.SearchMove()
        temp_prob = float(Prob_list[j])/float(len(temp_tower.PossibleMove))
        #print ("state = " ,state)
        #loop over possible moves, saved to next 
        for pair in temp_tower.PossibleMove:
            print("pair = ",pair)

            next_tower.reset()
            next_tower.set_status(state)

            print("status before =",next_tower.status) 
        
            next_tower.move_by_pair(pair)
            print("status after =",next_tower.status)
            print("status temp =",temp_tower.status)
            if next_tower.status in Stat_list_next:
                #print("yes")
                Prob_list_next[Stat_list_next.index(next_tower.status)] += temp_prob
            else:
                #print("no")
                #print (len(Stat_list_next))
                Stat_list_next.append(next_tower.status)
                Prob_list_next.append(temp_prob)
                #print (len(Stat_list_next))
com_list = []
for state in Stat_list_next:
    temp_tower = Initial
    temp_tower.set_status(state)
    com_list.append(temp_tower.COM())

ave = 0.0
px2 = 0.0
p2x2 = 0.0
#print (com_list)
#print(Prob_list_next)
for com, prob in zip(com_list,Prob_list_next):
    ave += com*prob
    px2 += prob*com**2
    p2x2 += (prob*com)**2
std = (px2-p2x2)**0.5
print (ave,std)

T= 0
1 1
next_tower [[ 1.  2.  3.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
pair =  [(0, 0), (1, 2)]
status before = [[ 1.  2.  3.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
status after = [[ 0.  2.  3.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
status temp = [[ 0.  2.  3.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
T= 1
1 1
next_tower [[ 0.  2.  3.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
pair =  [(0, 0), (1, 2)]
status before = [[ 0.  2.  3.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
status after = [[ 1.  2.  3.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
status temp = [[ 1.  2.  3.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
pair =  [(1, 2), (2, 2)]
status before = [[ 1.  2.  3.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
status after = [[ 1.  2.  3.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
status temp = [[ 1.  2.  3.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
T= 2
1 1
next_tower [[ 1.  2.  3.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
pair =  [(0, 0), (1, 2)]
status before = [[ 1.  2.  3.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
status after = [[ 0.  2.  3.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
status temp = [[ 0.  2.  3.]
 

In [377]:
#iteration for 10000 times

results = []
n = 100
T = 16
M3N3 = TowerMatrix(M=3,N=3)
#d = [M3N3,M3N3]

#d[1].move()
#print(d[1].status, d[0].status)

#e = TowerMatrix(M=3,N=3)

#if e in d:
#    print(a)
#ad = []
#pd = []
#ad.append(M3N3.status)
#pd.append(M3N3.COM())

#M3N3.move()
# ad.append(M3N3.status)
# pd.append(M3N3.COM())
# print(ad)
# print(pd)



# for i in range(100):
#     M3N3.move()
#     if M3N3.status in ad:
#         print (ad.index(M3N3.status))

for i in range(n):
   M3N3.reset()
   for j in range(T):
       M3N3.move()
       #print(M3N3.COM())
   results.append(M3N3.COM())
print(np.mean(results))  
print(np.std(results))


1.42666666667
0.316508732546


In [378]:
#test, successful
a =TowerMatrix(M=3,N=3)
b =TowerMatrix(M=3,N=3)
a.move()
print(a.COM())
print(b.COM())


1.16666666667
1.0


In [297]:
a = [1,2,3,4]
c = ["a","b","4","g"]
for t,y in zip(a,c):
    print(t,y)

1 a
2 b
3 4
4 g


In [128]:
c = [[(1,1),(2,2)],[(0,0),(0,0)],[(0,0),(0,1)],[(0,0),(0,2)]]

In [112]:
for i in range(100):
    print(random.SystemRandom().choice(c))

[(0, 0), (0, 0)]
[(0, 0), (0, 1)]
[(0, 0), (0, 0)]
[(0, 0), (0, 0)]
[(1, 1), (2, 2)]
[(0, 0), (0, 0)]
[(1, 1), (2, 2)]
[(0, 0), (0, 2)]
[(1, 1), (2, 2)]
[(1, 1), (2, 2)]
[(0, 0), (0, 0)]
[(0, 0), (0, 0)]
[(0, 0), (0, 2)]
[(0, 0), (0, 1)]
[(0, 0), (0, 2)]
[(0, 0), (0, 1)]
[(0, 0), (0, 2)]
[(1, 1), (2, 2)]
[(0, 0), (0, 1)]
[(0, 0), (0, 1)]
[(1, 1), (2, 2)]
[(1, 1), (2, 2)]
[(0, 0), (0, 2)]
[(0, 0), (0, 1)]
[(0, 0), (0, 0)]
[(0, 0), (0, 1)]
[(0, 0), (0, 1)]
[(1, 1), (2, 2)]
[(0, 0), (0, 2)]
[(0, 0), (0, 0)]
[(1, 1), (2, 2)]
[(1, 1), (2, 2)]
[(0, 0), (0, 0)]
[(0, 0), (0, 0)]
[(1, 1), (2, 2)]
[(0, 0), (0, 2)]
[(1, 1), (2, 2)]
[(1, 1), (2, 2)]
[(0, 0), (0, 0)]
[(0, 0), (0, 1)]
[(1, 1), (2, 2)]
[(0, 0), (0, 1)]
[(0, 0), (0, 2)]
[(0, 0), (0, 0)]
[(0, 0), (0, 2)]
[(0, 0), (0, 1)]
[(0, 0), (0, 0)]
[(0, 0), (0, 2)]
[(0, 0), (0, 1)]
[(1, 1), (2, 2)]
[(0, 0), (0, 2)]
[(0, 0), (0, 2)]
[(0, 0), (0, 2)]
[(1, 1), (2, 2)]
[(0, 0), (0, 2)]
[(0, 0), (0, 0)]
[(0, 0), (0, 1)]
[(1, 1), (2, 2)]
[(0, 0), (0, 0

In [113]:
def a():
    print (c)

In [114]:
a

<function __main__.a>

In [115]:
a()

[[(1, 1), (2, 2)], [(0, 0), (0, 0)], [(0, 0), (0, 1)], [(0, 0), (0, 2)]]


In [116]:
c

[[(1, 1), (2, 2)], [(0, 0), (0, 0)], [(0, 0), (0, 1)], [(0, 0), (0, 2)]]

In [142]:
a = 0.10000001000100
print(round(a,10))

0.10000001


In [143]:
sum(c)

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [185]:
a = [1,2,3,4,5]

In [145]:
sum(a)

15

In [146]:
np.sum(a)

15

In [166]:
for i in range(1,1):
    print(i)

In [199]:
print(np.random.choice(1))

0


In [273]:
a = np.zeros((6,6))

In [274]:
a[0]=[1,2,3,4,5,6]

In [276]:
a=[[1,2,3,4,5,6],[],[],[],[],[]]

In [277]:
a

[[1, 2, 3, 4, 5, 6], [], [], [], [], []]

In [278]:
a[0].remove(6)

In [279]:
a

[[1, 2, 3, 4, 5], [], [], [], [], []]

In [280]:
a[1].append(6)

In [281]:
a

[[1, 2, 3, 4, 5], [6], [], [], [], []]